# S.O.L.I.D.
## Content

1. [Overview](#overview)
2. [Single Responsibility Principle (SRP)](#single-responsibility-principle-srp)
3. [Open-Closed Principle (OCP)](#open-closed-principle-ocp)
4. [Liskov Substitution Principle (LSP)](#liskov-substitution-principle-lsp)
5. [Interface Segregation Principle (ISP)](#interface-segregation-principle-isp)
6. [Dependency Inversion Principle (DIP)](#dependency-inversion-principle-dip)
7. [Conclusion](#conclusion)
8. [Resources](#resources)

## Overview

SOLID is a set of five design principles that aim to make object-oriented software easier to understand, maintain, and extend. These principles were popularized by Robert C. Martin (Uncle Bob) in his 2000 paper "Design Principles and Design Patterns" and the acronym SOLID was coined a few years later.

The five SOLID principles are:

* **S**ingle Responsibility Principle (SRP)
* **O**pen-Closed Principle (OCP)
* **L**iskov Substitution Principle (LSP)
* **I**nterface Segregation Principle (ISP)
* **D**ependency Inversion Principle (DIP)

I'll be using the same example from [C# Clean Code: SOLID Principles – Dev.to](https://dev.to/moh_moh701/c-clean-code-solid-principles-51ed), so as the initial code we have the following:

In [1]:
public class OrderItem
{
  public string Name { get; set; }
  public decimal Price { get; set; }
  public int Quantity { get; set; }
}

public class Order 
{
  public int Id { get; set; }
  public List<OrderItem> Items { get; set; }
  public string CustomerType { get; set; } // 'Regular', 'Premium'

  public Order()
  {
    Items = new List<OrderItem>();
  }

  // Calculate order total
  public decimal GetTotal()
  {
    decimal total = 0;
    foreach (var item in Items)
    {
      total += item.Price * item.Quantity;
    }

    // Apply discount based on customer type
    if (CustomerType == "Premium")
    {
      total *= 0.9m; // 10% discount for premium customers
    }

    return total;
  }

  // Print order receipt
  public void PrintReceipt()
  {
    Console.WriteLine($"Order ID: {Id}");

    foreach (var item in Items)
    {
      Console.WriteLine($"{item.Name} - {item.Quantity} x {item.Price} = {item.Quantity * item.Price}");
    }
    
    Console.WriteLine($"Total: {GetTotal()}");
    }
}

## Single Responsibility Principle (SRP)

### Definition 

The Single Responsibility Principle states that a class should have only one reason to change.

### Use Case

`Order` class violates the SRP because it has two responsibilities, order calculation and receipt printing. To fix it, we can separate the responsibilities in two different classes.

### Solution

In [2]:
// This class only handles order calculation now.
public class Order
{
  public int Id { get; set; }
  public List<OrderItem> Items { get; set; }
  public string CustomerType { get; set; } // 'Regular', 'Premium'

  public Order()
  {
    Items = new List<OrderItem>();
  }

  // Calculate order total
  public decimal GetTotal()
  {
    decimal total = 0;
    foreach (var item in Items)
    {
      total += item.Price * item.Quantity;
    }

    // Apply discount based on customer type
    if (CustomerType == "Premium")
    {
      total *= 0.9m; // 10% discount for premium customers
    }

    return total;
  }
}

// This class only handles receipt printing now.
public class ReceiptPrinter
{
  // Print order receipt
  public void PrintReceipt(Order order)
  {
    Console.WriteLine($"Order ID: {order.Id}");

    foreach (var item in order.Items)
    {
      Console.WriteLine($"{item.Name} - {item.Quantity} x {item.Price} = {item.Quantity * item.Price}");
    }
    
    Console.WriteLine($"Total: {order.GetTotal()}");
  }
}

## Open-Closed Principle (OCP)

### Definition 

The Open-Closed Principle suggests that software entities should be open for extension but closed for modification.

### Use Case

The `GetTotal` method in our Order class has hardcoded discount logic for premium customers, which violates OCP. To fix it, we can move the discount logic outside the `Order` class, and pass it as parameter. This way, the discount functionality will be extendable without chaging the `Order` class code.

### Solution

In [3]:
// We declare IDiscount interface, and include a ApplyDiscount method, so we can have multiple classes implementing it for various kinds of discounts.
public interface IDiscount
{
  decimal ApplyDiscount(decimal total);
}

// This first class will handle no discount scenario.
public class NoDiscount : IDiscount
{
  public decimal ApplyDiscount(decimal total)
  {
    return total;
  }
}

// This second class will handle the discount for premium customers.
public class PremiumDiscount : IDiscount
{
  public decimal ApplyDiscount(decimal total)
  {
    return total * 0.9m; // 10% discount for premium customers
  }
}

// This class only handles order calculation now.
public class Order
{
  public int Id { get; set; }
  public List<OrderItem> Items { get; set; }
  public IDiscount Discount { get; set; } // Customer Type logic was removed, and instead IDiscount was added, so the discount can be handled from outside.

  // IDiscount interface is included in the constructor, so any kind of discount can passed from outside and we don't need to worry about how it's applied.
  public Order(IDiscount discount)
  {
    Items = new List<OrderItem>();
    Discount = discount;
  }

  // Calculate order total
  public decimal GetTotal()
  {
    decimal total = 0;
    foreach (var item in Items)
    {
      total += item.Price * item.Quantity;
    }

    // Condition to apply discount for premium customer was removed and instead the ApplyDiscount method is called from the Discount instance.
    return Discount.ApplyDiscount(total);
  }
}

## Liskov Substitution Principle (LSP)

### Definition 

The Liskov Substitution Principle states that subclasses should be substitutable for their base classes without altering the correctness of the program.

### Use Case

By introducing the `IDiscount` interface, we've already ensured that any class implementing `IDiscount` (like `PremiumDiscount`, `NoDiscount`, or `BirthdayDiscount`) can replace each other without breaking the functionality.

### Solution

In [ ]:
public interface IDiscount
{
  decimal ApplyDiscount(decimal total);
}

public class NoDiscount : IDiscount // This first class will handle no discount scenario.
{
  public decimal ApplyDiscount(decimal total)
  {
    return total;
  }
}

public class PremiumDiscount : IDiscount // This second class will handle the discount for premium customers.
{
  public decimal ApplyDiscount(decimal total)
  {
    return total * 0.9m; // 10% discount for premium customers
  }
}

public class BirthdayDiscount : IDiscount // This third class will handle the discount for customers' birthday.
{
  public decimal ApplyDiscount(decimal total)
  {
    return total * 0.95m; // 5% birthday discount
  }
}

public class Order
{
  public int Id { get; set; }
  public List<OrderItem> Items { get; set; }
  public IDiscount Discount { get; set; } // Customer Type logic was removed, and instead IDiscount was added, so the discount can be handled from outside.

  // IDiscount interface is included in the constructor, so any kind of discount can passed from outside and we don't need to worry about how it's applied.
  public Order(IDiscount discount)
  {
    Items = new List<OrderItem>();
    Discount = discount;
  }

  // Calculate order total
  public decimal GetTotal()
  {
    decimal total = 0;
    foreach (var item in Items)
    {
      total += item.Price * item.Quantity;
    }

    // Condition to apply discount for premium customer was removed and instead the ApplyDiscount method is called from the Discount instance.
    return Discount.ApplyDiscount(total);
  }
}

var birthdayOrder = new Order(new BirthdayDiscount()); // Any discount class can now be used in place of another, following LSP
birthdayOrder.Items.Add(new OrderItem { Name = "Cake", Price = 20, Quantity = 1 });
birthdayOrder.Items.Add(new OrderItem { Name = "Ice Cream", Price = 5, Quantity = 2 });
Console.WriteLine(birthdayOrder.GetTotal());

28.50


## Interface Segregation Principle (ISP)

### Definition 

The Interface Segregation Principle suggests that clients should not be forced to depend on interfaces they don't use.

### Use Case

Instead of creating one large interface (e.g., `IOrderManager`), it's better to break it into smaller, more focused interfaces. Let's apply ISP by splitting responsibilities into smaller interfaces for orders and receipt printing.

### Solution

In [ ]:
// Adding this new interface, with "GetTotal" as the only required method to be implemented .
public interface IOrder
{
  decimal GetTotal();
}

// Adding this new interface, with "PrintReceipt" as the only required method to be implemented .
public interface IReceiptPrinter
{
  void PrintReceipt(Order order);
}

// Order class is only implementing required methods from IOrder interface.
public class Order : IOrder 
{
  public int Id { get; set; }
  public List<OrderItem> Items { get; set; }
  public IDiscount Discount { get; set; } // Customer Type logic was removed, and instead IDiscount was added, so the discount can be handled from outside.

  // IDiscount interface is included in the constructor, so any kind of discount can passed from outside and we don't need to worry about how it's applied.
  public Order(IDiscount discount)
  {
    Items = new List<OrderItem>();
    Discount = discount;
  }

  // Calculate order total
  public decimal GetTotal()
  {
    decimal total = 0;
    foreach (var item in Items)
    {
      total += item.Price * item.Quantity;
    }

    // Condition to apply discount for premium customer was removed and instead the ApplyDiscount method is called from the Discount instance.
    return Discount.ApplyDiscount(total);
  }
}

// ReceiptPrinter class is only implementing required methods from IReceiptPrinter interface.
public class ReceiptPrinter : IReceiptPrinter
{
  // Print order receipt
  public void PrintReceipt(Order order)
  {
    Console.WriteLine($"Order ID: {order.Id}");

    foreach (var item in order.Items)
    {
      Console.WriteLine($"{item.Name} - {item.Quantity} x {item.Price} = {item.Quantity * item.Price}");
    }
    
    Console.WriteLine($"Total: {order.GetTotal()}");
  }
}

## Dependency Inversion Principle (DIP)

### Definition 

The Dependency Inversion Principle states that high-level modules should depend on abstractions, not on concrete implementations.

### Use Case

We're already using abstractions when making `Order` class depending on an abstraction for discounts `IDiscount`, instead of concrete implementations like `PremiumDiscount` or `NoDiscount`.

### Solution

In [ ]:
public interface IOrder
{
  decimal GetTotal();
}

public class Order : IOrder 
{
  public int Id { get; set; }
  public List<OrderItem> Items { get; set; }
  private readonly IDiscount _discount; // We can change the access modifier to private and set it to readonly, so this parameter is only passed in the constructor.

  // The discount will depend on an abstraction for discounts, IDiscount, instead of concrete implementations like PremiumDiscount or NoDiscount.
  public Order(IDiscount discount)
  {
    Items = new List<OrderItem>();
    _discount = discount;
  }

  // Calculate order total
  public decimal GetTotal()
  {
    decimal total = 0;
    foreach (var item in Items)
    {
      total += item.Price * item.Quantity;
    }

    return _discount.ApplyDiscount(total);
  }
}

## Conclusion

By applying the SOLID principles to a simple Customer Order System example, we refactored the code to become more maintainable, scalable, and flexible. Each principle brings a unique benefit:

- **Single Responsibility Principle:** Makes each class focused on a single task, improving clarity and maintainability.
- **Open-Closed Principle:** Allows extending functionality without modifying existing code, reducing the risk of introducing bugs.
- **Liskov Substitution Principle:** Ensures that subclasses can be used in place of their base classes, preserving correctness.
- **Interface Segregation Principle:** Promotes the use of smaller, more focused interfaces, reducing unnecessary dependencies.
- **Dependency Inversion Principle:** Encourages classes to depend on abstractions rather than concrete implementations, improving flexibility.

## Resources

- *Clean Architecture: A Craftsman's Guide to Software Structure and Design*, by Robert C. Martin
- [C# Clean Code: SOLID Principles – Dev.to](https://dev.to/moh_moh701/c-clean-code-solid-principles-51ed)